In [1]:
%matplotlib qt

In [2]:
import mne
from pprint import pprint
import json
import csv
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from pathlib import Path
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)

In [37]:
# Use the .fif files produced from kit2fiff packages
#root_data_path = Path("/Users/neuroling/Downloads/20250610_EEG-Demo")
root_data_path = Path("/Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG")
print(root_data_path)

# new = new gel ; original: old gel
#EEG_cntFile = root_data_path / Path('eeg_20250610_test_new_Notched.cnt') #eeg_20250610_test_original_noNotch.cnt
#LDT_raw =  mne.io.read_raw_kit(LDT_confile, stim = [195], preload=True) #, slope = '+') #slope=將3個stim_channel的訊號全部加成,讓訊號增強
                                                                #把原始的KIT資料讀進
EEG_cntFile = root_data_path / Path('20251021_CEOE_P20.cnt')

#LDT_raw.plot()
#events = mne.find_events(LDT_raw, stim_channel='STI 014') #將集合而成的channel命名為STI 014

MMN_raw = mne.io.read_raw_cnt(EEG_cntFile, eog="header", preload=True)
MMN_raw.plot() # Trigger 1 = Close Eye (CE); Trigger 10 = Open Eye (OE)
n_MMN_raw = MMN_raw.copy()
#n_MMN_filtered = n_MMN_raw.notch_filter(freqs=50, picks="all")
#n_MMN_raw.plot()
#n_MMN_filtered.plot()
print(n_MMN_raw.info)

"""
# Calculate the power spectrum density to examine the power in each frequency.
n_MMN_raw.compute_psd(fmin=1, fmax=100).plot()#.(fmax=100)

# Save the cnt into .fif for a easier further analysis
fif_DIR = root_data_path / 'rawfif_all'
#uncut_fif_DIR = fif_DIR / 'Uncut_rawfif'
fif_DIR.mkdir(exist_ok=True)

n_MMN_raw.save(fif_DIR / Path('20251021_SSAEP-1_P20.fif'), overwrite=True)
"""

## For cuting CE & OE
events_ann_ndArray = mne.events_from_annotations(n_MMN_raw)[0]
print(events_ann_ndArray)
print(type(events_ann_ndArray))
print(len(events_ann_ndArray))

# Set the CEOE based on triggers (Trigger 1 = Close Eye (CE); Trigger 10 = Open Eye (OE))
events_id_DICT = {'1': 1, '10': 2}
raw_sfreq = n_MMN_raw.info['sfreq'] # supposedly to be 1000 Hz
#fifname_LIST = ["eyeClose_3min", "eyeOpen_3min"]

CE_start_ndArray = events_ann_ndArray[0]
OE_start_ndArray = events_ann_ndArray[1]

#print(CE_start_ndArray[2])

CE_raw_fif = n_MMN_raw.copy().crop((CE_start_ndArray[0]/int(raw_sfreq))-0.2, (CE_start_ndArray[0]/int(raw_sfreq))+ 3.2)
OE_raw_fif = n_MMN_raw.copy().crop((OE_start_ndArray[0]/int(raw_sfreq))-0.2, (OE_start_ndArray[0]/int(raw_sfreq))+ 3.2)

CE_raw_fif.save(fif_DIR / Path('CE_P20.fif'), overwrite=True)
OE_raw_fif.save(fif_DIR / Path('OE_P20.fif'), overwrite=True)

/Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG
Reading 0 ... 537087  =      0.000 ...   524.499 secs...


/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_44779/3581649047.py:15: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  MMN_raw = mne.io.read_raw_cnt(EEG_cntFile, eog="header", preload=True)


Using pyopengl with version 3.1.1a1
<Info | 10 non-empty values
 bads: 1 items (Trigger)
 ch_names: Fz_1, Trigger
 chs: 1 EOG, 1 EEG
 custom_ref_applied: False
 dig: 5 items (3 Cardinal, 2 EEG)
 highpass: 0.0 Hz
 lowpass: 512.0 Hz
 meas_date: unspecified
 nchan: 2
 projs: []
 sfreq: 1024.0 Hz
 subject_info: 5 items (dict)
>
Used Annotations descriptions: ['1', '10']
[[ 13207      0      1]
 [234752      0      2]]
<class 'numpy.ndarray'>
2
Writing /Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/CE_P20.fif
Closing /Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/CE_P20.fif
[done]
Writing /Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/OE_P20.fif
Closing /Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/OE_P20.fif
[done]


/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_44779/3581649047.py:54: RuntimeWarning: This filename (/Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/CE_P20.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  CE_raw_fif.save(fif_DIR / Path('CE_P20.fif'), overwrite=True)
/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_44779/3581649047.py:55: RuntimeWarning: This filename (/Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/OE_P20.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  OE_raw_fif.save(fif_DIR / Path('OE_P20.fif'), overwrite=True)


Channels marked as bad:
['Trigger']


In [27]:
n_MMN_raw.ch_names # Where are the annotations

['Fz_1', 'Trigger']

In [ ]:
def set_psd_layout(fif, datapath):
    fif = mne.io.read_raw_fif(fif_DIR / Path('CE_AGN.fif'), preload=True)

In [16]:
# Use the .fif files produced from kit2fiff packages
#root_data_path = Path("/Users/neuroling/Downloads/20250610_EEG-Demo")
root_data_path = Path("/Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG")
print(root_data_path)
fif_DIR = root_data_path / 'rawfif_all'

# noNotch
CE_AGN_fif = mne.io.read_raw_fif(fif_DIR / Path('CE_AGN_raw.fif'), preload=True)
OE_AGN_fif = mne.io.read_raw_fif(fif_DIR / Path('OE_AGN_raw.fif'), preload=True)
CE_P20_fif = mne.io.read_raw_fif(fif_DIR / Path('CE_P20_raw.fif'), preload=True)
OE_P20_fif = mne.io.read_raw_fif(fif_DIR / Path('OE_P20_raw.fif'), preload=True)

#(X)CE_AGN_fif.compute_psd(fmin=1, fmax=100).plot()
#(X)CE_P20_fif.compute_psd(fmin=1, fmax=100).plot()

# Check whether the ch exists
print(CE_AGN_fif.info['ch_names'])
print(CE_AGN_fif.get_channel_types())
CE_AGN_fif.rename_channels({'Fz_1': 'Fz_1'})
CE_AGN_fif.set_channel_types({'Fz_1': 'eeg'})

OE_AGN_fif.rename_channels({'Fz_1': 'Fz_1'})
OE_AGN_fif.set_channel_types({'Fz_1': 'eeg'})

CE_P20_fif.rename_channels({'Fz_1': 'Fz_1'})
CE_P20_fif.set_channel_types({'Fz_1': 'eeg'})

OE_P20_fif.rename_channels({'Fz_1': 'Fz_1'})
OE_P20_fif.set_channel_types({'Fz_1': 'eeg'})

CE_P20_fif.compute_psd(fmin=1, fmax=100).plot()

"""
# NOPE FOR THESE
merged_CE_LIST = [CE_AGN_fif, CE_P20_fif]
merged_OE_LIST = [OE_AGN_fif, OE_P20_fif]
#merged_MMN_LIST.plot()

merged_CE = mne.concatenate_raws(merged_CE_LIST)
#merged_OE = mne.concatenate_raws(merged_OE_LIST)
merged_CE.compute_psd(fmin=1, fmax=100).plot()
#merged_OE.compute_psd(fmin=1, fmax=100).plot()
"""

# Make the epoch alongside
## For cuting CE & OE
events_CE_ndArray = mne.events_from_annotations(CE_P20_fif)[0]
events_OE_ndArray = mne.events_from_annotations(OE_P20_fif)[0]
print(events_CE_ndArray)
print(type(events_CE_ndArray))
print(len(events_CE_ndArray))

CE_events = events_CE_ndArray
OE_events = events_OE_ndArray

events_CE_id_DICT = {'1': 1}
CE_sfreq = CE_AGN_fif.info['sfreq'] # supposedly to be 1000 Hz

events_OE_id_DICT = {'1': 1}
OE_sfreq = OE_AGN_fif.info['sfreq'] # supposedly to be 1000 Hz

tmin = -0.1         								# pre stimulis interval (in seconds) #
tmax = 3 # for 3 seconds     
baseline = (None, 0)

CE_epoch = mne.Epochs(CE_P20_fif, CE_events, events_CE_id_DICT, tmin, tmax, baseline=baseline, preload = True)
OE_epoch = mne.Epochs(OE_P20_fif, OE_events, events_OE_id_DICT, tmin, tmax, baseline=baseline, preload = True)
print(CE_epoch.info)
#CE_epoch.save(fif_DIR / Path('CE_P20_epoch.fif'), overwrite=True)
#OE_epoch.save(fif_DIR / Path('OE_P20_epoch.fif'), overwrite=True)

/Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG
Opening raw data file /Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/CE_AGN_raw.fif...
    Range : 8197 ... 11679 =      8.005 ...    11.405 secs
Ready.
Reading 0 ... 3482  =      0.000 ...     3.400 secs...
Opening raw data file /Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/OE_AGN_raw.fif...
    Range : 206425 ... 209907 =    201.587 ...   204.987 secs
Ready.
Reading 0 ... 3482  =      0.000 ...     3.400 secs...
Opening raw data file /Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/CE_P20_raw.fif...
    Range : 13002 ... 16484 =     12.697 ...    16.098 secs
Ready.
Reading 0 ... 3482  =      0.000 ...     3.400 secs...
Opening raw data file /Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/OE_P20_raw.fif...
    Range : 234547 ... 238029 =    229.050 ...   232.450 secs
Ready.
Reading 0 ... 3482  =      0.000 ...     3.400 secs...
['Fz_1', 'Trigger']
['eog', 'eeg']
Effective window size : 2.000 (s)
U

In [17]:
## Eye Open All freq bands
import numpy as np
import matplotlib.pyplot as plt
from mne.filter import filter_data
from scipy.signal import hilbert  # optional: smooth envelope

# ---------- Settings ----------
channel = 'Fz_1'
epoch_idx = 0                     # which epoch to visualize
bands = {
    'δ wave (0.5–4 Hz)': (0.5, 4),
    'θ wave (4–8 Hz)'  : (4, 8),
    'α wave (8–12 Hz)' : (8, 12),
    'β wave (13–30 Hz)': (13, 30),
}
use_envelope = False              # set True to plot amplitude envelopes
y_offset = 3.0                    # vertical spacing between traces

# ---------- Get one epoch & channel ----------
data = OE_epoch.get_data(picks=channel)[epoch_idx, 0, :]   # shape: (n_times,)
times = OE_epoch.times                                     # seconds
sfreq = OE_epoch.info['sfreq']

# ---------- Band-pass per band ----------
filtered = {}
for name, (fmin, fmax) in bands.items():
    x = filter_data(data.copy(), sfreq=sfreq, l_freq=fmin, h_freq=fmax,
                    method='fir', phase='zero')  # linear-phase for visualization
    if use_envelope:
        x = np.abs(hilbert(x))                   # optional amplitude envelope
    # normalize to a.u. for nicer stacking
    x = x / (np.std(x) + 1e-12)
    filtered[name] = x

# ---------- Plot like the paper (stacked) ----------
plt.figure(figsize=(10, 6))
colors = ['black', 'red', 'green', 'blue']
for i, (name, col) in enumerate(zip(filtered.keys(), colors)):
    offset = y_offset * (len(bands) - i - 1)
    plt.plot(times, filtered[name] + offset, color=col, linewidth=1.5, label=name)

plt.xlabel('Time (s)')
plt.ylabel('Signal (a.u.)')
plt.title(f'Band‑limited oscillations at {channel} (Eyes Open, epoch {epoch_idx})')
plt.yticks([])               # hide y ticks to match “a.u.” style
plt.grid(True, axis='x')
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()
# plt.savefig('stacked_waves_eyeC.png', dpi=300)


Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 6759 samples (6.601 s)

Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter 

/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_79433/2265328449.py:27: RuntimeWarning: filter_length (6759) is longer than the signal (3175), distortion is likely. Reduce filter length or filter a longer signal.
  x = filter_data(data.copy(), sfreq=sfreq, l_freq=fmin, h_freq=fmax,
2026-02-10 14:48:34.335 python[79433:19234896] +[CATransaction synchronize] called within transaction


In [18]:
## Eye Close All freq bands
import numpy as np
import matplotlib.pyplot as plt
from mne.filter import filter_data
from scipy.signal import hilbert  # optional: smooth envelope

# ---------- Settings ----------
channel = 'Fz_1'
epoch_idx = 0                     # which epoch to visualize
bands = {
    'δ wave (0.5–4 Hz)': (0.5, 4),
    'θ wave (4–8 Hz)'  : (4, 8),
    'α wave (8–12 Hz)' : (8, 12),
    'β wave (13–30 Hz)': (13, 30),
}
use_envelope = False              # set True to plot amplitude envelopes
y_offset = 3.0                    # vertical spacing between traces

# ---------- Get one epoch & channel ----------
data = CE_epoch.get_data(picks=channel)[epoch_idx, 0, :]   # shape: (n_times,)
times = CE_epoch.times                                     # seconds
sfreq = CE_epoch.info['sfreq']

# ---------- Band-pass per band ----------
filtered = {}
for name, (fmin, fmax) in bands.items():
    x = filter_data(data.copy(), sfreq=sfreq, l_freq=fmin, h_freq=fmax,
                    method='fir', phase='zero')  # linear-phase for visualization
    if use_envelope:
        x = np.abs(hilbert(x))                   # optional amplitude envelope
    # normalize to a.u. for nicer stacking
    x = x / (np.std(x) + 1e-12)
    filtered[name] = x

# ---------- Plot like the paper (stacked) ----------
plt.figure(figsize=(10, 6))
colors = ['black', 'red', 'green', 'blue']
for i, (name, col) in enumerate(zip(filtered.keys(), colors)):
    offset = y_offset * (len(bands) - i - 1)
    plt.plot(times, filtered[name] + offset, color=col, linewidth=1.5, label=name)

plt.xlabel('Time (s)')
plt.ylabel('Signal (a.u.)')
plt.title(f'Band‑limited oscillations at {channel} (Eyes Closed, epoch {epoch_idx})')
plt.yticks([])               # hide y ticks to match “a.u.” style
plt.grid(True, axis='x')
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()
# plt.savefig('stacked_waves_eyeC.png', dpi=300)


Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 6759 samples (6.601 s)

Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter 

/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_79433/3771142569.py:27: RuntimeWarning: filter_length (6759) is longer than the signal (3175), distortion is likely. Reduce filter length or filter a longer signal.
  x = filter_data(data.copy(), sfreq=sfreq, l_freq=fmin, h_freq=fmax,
2026-02-10 14:49:10.057 python[79433:19234896] +[CATransaction synchronize] called within transaction


In [13]:
## Eye Open & Close comparison under 4 diff freq bands
import numpy as np
import matplotlib.pyplot as plt
from mne.filter import filter_data

channel = 'Fz_1'
epoch_idx = 0
bands = {
    'δ (0.5–4 Hz)' : (0.5, 4),
    'θ (4–8 Hz)'   : (4, 8),
    'α (8–12 Hz)'  : (8, 12),
    'β (13–30 Hz)' : (13, 30),
}

# Extract one epoch from each condition
x_O = OE_epoch.get_data(picks=channel)[epoch_idx, 0, :]
x_C = CE_epoch.get_data(picks=channel)[epoch_idx, 0, :]
times = OE_epoch.times
sfreq = OE_epoch.info['sfreq']

n = len(bands)
plt.figure(figsize=(12, 2.2*n))

for i, (name, (fmin, fmax)) in enumerate(bands.items(), start=1):
    yO = filter_data(x_O.copy(), sfreq, fmin, fmax, method='fir', phase='zero')
    yC = filter_data(x_C.copy(), sfreq, fmin, fmax, method='fir', phase='zero')

    # normalize each to a.u. for visual comparability
    yO = yO / (np.std(yO) + 1e-12)
    yC = yC / (np.std(yC) + 1e-12)

    ax = plt.subplot(n, 1, i)
    ax.plot(times, yC, label='Eyes Closed', linewidth=1.5)
    ax.plot(times, yO, label='Eyes Open', linewidth=1.2, alpha=0.9)
    ax.set_title(name)
    ax.set_ylabel('a.u.')
    ax.grid(True, axis='x')
    if i == n:
        ax.set_xlabel('Time (s)')
    if i == 1:
        ax.legend(loc='upper right')

plt.tight_layout()
plt.show()
# plt.savefig('overlay_OE_CE_per_band.png', dpi=300)

Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 6759 samples (6.601 s)

Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filte

/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_79433/2765535550.py:25: RuntimeWarning: filter_length (6759) is longer than the signal (3175), distortion is likely. Reduce filter length or filter a longer signal.
  yO = filter_data(x_O.copy(), sfreq, fmin, fmax, method='fir', phase='zero')
/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_79433/2765535550.py:26: RuntimeWarning: filter_length (6759) is longer than the signal (3175), distortion is likely. Reduce filter length or filter a longer signal.
  yC = filter_data(x_C.copy(), sfreq, fmin, fmax, method='fir', phase='zero')
2026-02-10 14:39:20.995 python[79433:19234896] +[CATransaction synchronize] called within transaction


## MMN Analysis (Below)

In [3]:
# Use the .fif files produced from kit2fiff packages
root_data_path = Path("/Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG")
print(root_data_path)

EEG_cntFile = root_data_path / Path('20251021_SSAEP-1_AGN.cnt')
MMN_raw = mne.io.read_raw_cnt(EEG_cntFile, eog="header", preload=True)
MMN_raw.plot() # Trigger 1 = Close Eye (CE); Trigger 10 = Open Eye (OE)
n_MMN_raw = MMN_raw.copy()

#n_MMN_filtered = n_MMN_raw.notch_filter(freqs=50, picks="all")
#n_MMN_raw.plot()
#n_MMN_filtered.plot()
print(n_MMN_raw.info)
"""
# Calculate the power spectrum density to examine the power in each frequency.
#n_MMN_raw.compute_psd(fmin=1, fmax=100).plot()#.(fmax=100)

# Save the cnt into .fif for a easier further analysis
fif_DIR = root_data_path / 'rawfif_all'
#uncut_fif_DIR = fif_DIR / 'Uncut_rawfif'
fif_DIR.mkdir(exist_ok=True)

n_MMN_raw.save(fif_DIR / Path('20251021_SSAEP-1_AGN.fif'), overwrite=True)
"""

/Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG


/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_2926/1937259273.py:6: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  MMN_raw = mne.io.read_raw_cnt(EEG_cntFile, eog="header", preload=True)


Reading 0 ... 228095  =      0.000 ...   222.749 secs...
Using qt as 2D backend.
Using pyopengl with version 3.1.1a1
<Info | 10 non-empty values
 bads: 1 items (Trigger)
 ch_names: Fz_1, Trigger
 chs: 1 EOG, 1 EEG
 custom_ref_applied: False
 dig: 5 items (3 Cardinal, 2 EEG)
 highpass: 0.0 Hz
 lowpass: 512.0 Hz
 meas_date: unspecified
 nchan: 2
 projs: []
 sfreq: 1024.0 Hz
 subject_info: 5 items (dict)
>


"\n# Calculate the power spectrum density to examine the power in each frequency.\n#n_MMN_raw.compute_psd(fmin=1, fmax=100).plot()#.(fmax=100)\n\n# Save the cnt into .fif for a easier further analysis\nfif_DIR = root_data_path / 'rawfif_all'\n#uncut_fif_DIR = fif_DIR / 'Uncut_rawfif'\nfif_DIR.mkdir(exist_ok=True)\n\nn_MMN_raw.save(fif_DIR / Path('20251021_SSAEP-1_AGN.fif'), overwrite=True)\n"

Channels marked as bad:
['Trigger']


In [51]:
# Creating the new epochs
fif_DIR = root_data_path / 'rawfif_all'
MMN_raw_fif = mne.io.read_raw_fif(fif_DIR / Path('20251021_SSAEP-1_P20.fif'), preload=True)

# Set the event markers according to the annotations
MMN_eventsArray = mne.events_from_annotations(MMN_raw_fif)[0]
MMN_events = MMN_eventsArray #mne.find_events(raw_ICAed, stim_channel='STI 014')  #just checking what stim_channel documenting what.
print(MMN_events)
#print(len(MMN_events))

# Turn all the events into lable 1
MMN_events[:, 2] = 1
print(MMN_events)

MMN_event_id = {'MMN':1}  

tmin = -0.1         								# pre stimulis interval (in seconds) #
tmax = 1          								# post stimulus interval #

baseline = (None, 0)  # tmin ~ 0 , -0.1s-0s   

#MMN_raw_fif.filter(0, 70, phase= 'zero-double').resample(500)
#raw_MMN_fif.resample(500)
#MMN_raw_fif.notch_filter(freqs=60)
#OE_epoch = mne.Epochs(OE_P20_fif, OE_events, events_OE_id_DICT, tmin, tmax, baseline=baseline, preload = True)
n_epoch_MMN = mne.Epochs(MMN_raw_fif, MMN_events, MMN_event_id, tmin, tmax, baseline=baseline, preload=True, event_repeated='drop') #, MMN_event_id
print(n_epoch_MMN.info)
n_epoch_MMN.save(fif_DIR / Path('MMN_P20_epochs_filtered.fif'), overwrite=True)
n_epoch_MMN

Opening raw data file /Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/20251021_SSAEP-1_P20.fif...
    Range : 0 ... 212991 =      0.000 ...   207.999 secs
Ready.
Reading 0 ... 212991  =      0.000 ...   207.999 secs...
Used Annotations descriptions: ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '255', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '6', '7', '8', '9']
[[ 16856      0      2]
 [ 18958      0      2]
 [ 19336      0     13]
 [ 21391      0     13]
 [ 21865      0     25]
 [ 24906      0     36]
 [ 27844      0     47]
 [ 30320      0     54]
 [ 32800      0     55]
 [ 34856      0     55]
 [ 35433      0     56]
 [ 38474      0     57]
 [ 41410      0      3]
 [ 43938      0      4]
 [ 45994      0      4]
 [ 47034      0      5]
 [ 49090      0      5

/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_2926/1399378972.py:3: RuntimeWarning: This filename (/Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/20251021_SSAEP-1_P20.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  MMN_raw_fif = mne.io.read_raw_fif(fif_DIR / Path('20251021_SSAEP-1_P20.fif'), preload=True)
/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_2926/1399378972.py:28: RuntimeWarning: This filename (/Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/MMN_P20_epochs_filtered.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  n_epoch_MMN.save(fif_DIR / Path('MMN_P20_epochs_filtered.fif'), overwrite=True)


Number of events,39
Events,MMN: 39
Time range,-0.100 – 1.000 s
Baseline,-0.100 – 0.000 s


In [66]:
##All freq bands
import numpy as np
import matplotlib.pyplot as plt
from mne.filter import filter_data
from scipy.signal import hilbert  # optional: smooth envelope

# ---------- Settings ----------
channel = 'Fz_1'
epoch_idx = 0                     # which epoch to visualize
bands = {
    'δ wave (0.5–4 Hz)': (0.5, 4),
    'θ wave (4–8 Hz)'  : (4, 8),
    'α wave (8–12 Hz)' : (8, 12),
    'β wave (13–30 Hz)': (13, 30),
}
use_envelope = False              # set True to plot amplitude envelopes
y_offset = 3.0                    # vertical spacing between traces

# ---------- Get one epoch & channel ----------
data = n_epoch_MMN.get_data(picks=channel)[epoch_idx, 0, :]   # shape: (n_times,)
times = n_epoch_MMN.times                                     # seconds
sfreq = n_epoch_MMN.info['sfreq']

# ---------- Band-pass per band ----------
filtered = {}
for name, (fmin, fmax) in bands.items():
    x = filter_data(data.copy(), sfreq=sfreq, l_freq=fmin, h_freq=fmax,
                    method='fir', phase='zero')  # linear-phase for visualization
    if use_envelope:
        x = np.abs(hilbert(x))                   # optional amplitude envelope
    # normalize to a.u. for nicer stacking
    x = x / (np.std(x) + 1e-12)
    filtered[name] = x

# ---------- Plot like the paper (stacked) ----------
plt.figure(figsize=(10, 6))
colors = ['black', 'red', 'green', 'blue']
for i, (name, col) in enumerate(zip(filtered.keys(), colors)):
    offset = y_offset * (len(bands) - i - 1)
    plt.plot(times, filtered[name] + offset, color=col, linewidth=1.5, label=name)

plt.xlabel('Time (s)')
plt.ylabel('Signal (a.u.)')
plt.title(f'Band‑limited oscillations at {channel} (MMN P20, epoch {epoch_idx})')
plt.yticks([])               # hide y ticks to match “a.u.” style
plt.grid(True, axis='x')
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()
# plt.savefig('stacked_waves_eyeC.png', dpi=300)


Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 3301 samples (6.602 s)

Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter 

/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_2926/2808434297.py:27: RuntimeWarning: filter_length (3301) is longer than the signal (551), distortion is likely. Reduce filter length or filter a longer signal.
  x = filter_data(data.copy(), sfreq=sfreq, l_freq=fmin, h_freq=fmax,
/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_2926/2808434297.py:27: RuntimeWarning: filter_length (825) is longer than the signal (551), distortion is likely. Reduce filter length or filter a longer signal.
  x = filter_data(data.copy(), sfreq=sfreq, l_freq=fmin, h_freq=fmax,
/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_2926/2808434297.py:27: RuntimeWarning: filter_length (825) is longer than the signal (551), distortion is likely. Reduce filter length or filter a longer signal.
  x = filter_data(data.copy(), sfreq=sfreq, l_freq=fmin, h_freq=fmax,


In [67]:
fif_DIR = root_data_path / 'rawfif_all'
channel = 'Fz_1'

MMN_P20_epochs = mne.read_epochs(fif_DIR / Path('MMN_P20_epochs.fif'), preload=True)
MMN_AGN_epochs = mne.read_epochs(fif_DIR / Path('MMN_AGN_epochs.fif'), preload=True)
#MMN_P20_epochs

# P20 Gel
MMN_P20_epochs.apply_baseline(baseline=(None, 0))
MMN_P20_ERP = MMN_P20_epochs.average(picks=channel)

# AGN Gel
MMN_AGN_epochs.apply_baseline(baseline=(None, 0))
MMN_AGN_ERP = MMN_AGN_epochs.average(picks=channel)


"""
# Plot mmn differences
tmp = [MMN_AGN_ERP, MMN_P20_ERP] #, evoked_deviant_3]
#mmn_compare = mne.combine_evoked([MMN_AGN_ERP, MMN_P20_ERP], weights=[1, -1])  # dev - std
colors = 'red', 'blue' #, 'green'
#mne.viz.plot_evoked_topo(tmp, color = colors, title = 'MMN', ylim = dict(eeg=[10, -10]))
mmn_fig = mne.viz.plot_compare_evokeds(tmp, colors=colors, picks=channel) #, axes = 'topo')
"""
# Force MNE to treat the data as EEG so it allows plotting
MMN_AGN_ERP.set_channel_types({channel: 'eeg'})
MMN_P20_ERP.set_channel_types({channel: 'eeg'})

# 1. Create a dictionary.
# The keys ('AGN', 'P20') will automatically become your Legend labels.
evokeds_dict = {
    'AGN': MMN_AGN_ERP,
    'P20': MMN_P20_ERP
}

# 2. Define colors mapping to those specific keys
colors_dict = {
    'AGN': 'red',
    'P20': 'blue'
}

# 3. Plot
# We use picks='all' because you already selected the single channel 'Fz_1' 
# when you ran .average(picks=channel) earlier.
mmn_fig = mne.viz.plot_compare_evokeds(
    evokeds_dict,
    colors=colors_dict,
    picks='all',       # Plots whatever channel is in the data
    title=f'MMN Comparison at {channel}',
    show_sensors=False # Optional: hides the small head map since it's just 1 channel
)

Reading /Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/MMN_P20_epochs.fif ...
    Found the data of interest:
        t =     -99.61 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
83 matching events found
No baseline correction applied
0 projection items activated
Reading /Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/MMN_AGN_epochs.fif ...
    Found the data of interest:
        t =     -99.61 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
84 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)


/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_2926/1174202747.py:4: RuntimeWarning: This filename (/Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/MMN_P20_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  MMN_P20_epochs = mne.read_epochs(fif_DIR / Path('MMN_P20_epochs.fif'), preload=True)
/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_2926/1174202747.py:5: RuntimeWarning: This filename (/Volumes/DH_4GB/2025R605_EEG_Demo/20251021_EEG/rawfif_all/MMN_AGN_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  MMN_AGN_epochs = mne.read_epochs(fif_DIR / Path('MMN_AGN_epochs.fif'), preload=True)
2026-02-11 17:51:13.090 python[2926:39048] +[CATransaction synchronize] called within transaction


MMN_AGN_ERP.filter(h_freqs=60) #(l_freq=None, h_freq=50)
MMN_AGN_ERP.plot(
    picks='all', 
    spatial_colors=False,  # Make the line black instead of color-coded by location
    gfp=False              # Turn off Global Field Power (irrelevant for 1 channel)
)

In [75]:
# 1. Filter the ERPs to remove the noise
high_freq = 50
MMN_AGN_ERP.filter(l_freq=None, h_freq=high_freq)
MMN_P20_ERP.filter(l_freq=None, h_freq=high_freq)

# 2. Plot the comparison again (Standard ERP view)
mne.viz.plot_compare_evokeds(
    {'AGN': MMN_AGN_ERP, 'P20': MMN_P20_ERP},
    colors={'AGN': 'red', 'P20': 'blue'},
    picks='all',
    title='MMN Comparison (Low-pass filtered @ 50Hz)'
)

Setting up low-pass filter at 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 271 samples (0.265 s)

Setting up low-pass filter at 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 271 samples (0.265 s)



[<Figure size 1600x1200 with 2 Axes>]

2026-02-11 17:55:09.302 python[2926:39048] +[CATransaction synchronize] called within transaction
2026-02-11 17:55:13.604 python[2926:39048] +[CATransaction synchronize] called within transaction
2026-02-11 17:55:14.574 python[2926:39048] +[CATransaction synchronize] called within transaction
2026-02-11 17:55:14.840 python[2926:39048] +[CATransaction synchronize] called within transaction


In [71]:
#(XWe are not using this for now) 2) ERP waveforms (standard, deviant, difference) with 100–250 ms window shaded + ERP‑SNR in legend
import numpy as np
import matplotlib.pyplot as plt
import mne

# ----------- CONFIG -----------
channel     = 'Fz_1'
std_key     = 'standard'
dev_key     = 'deviant_2'   # or your deviant label
mmn_window  = (0.10, 0.25)  # seconds (adjust to your paradigm)

# Input 2 type of data
epoch_DIR = fif_DIR / 'MMN_epochs'
# Original
orig_MMN_merged_epoch_fif = mne.read_epochs(epoch_DIR / Path('orig_MMN_merged_noFiltSamp_epochs.fif'), preload=True)
#orig_MMN_epoch_psd = orig_MMN_merged_epoch_fif.compute_psd(fmin=1, fmax=70).plot()
# New
#new_MMN_merged_epoch_fif = mne.read_epochs(epoch_DIR / Path('new_MMN_merged_noFiltSamp_epochs.fif'), preload=True)
#new_MMN_epoch_psd = new_MMN_merged_epoch_fif.compute_psd(fmin=1, fmax=70).plot()
"""
# Use full epoch (already baseline-corrected above)
epochs_std_full = orig_MMN_merged_epoch_fif[std_key].copy().baseline((None, 0))
epochs_dev_full = orig_MMN_merged_epoch_fif[dev_key].copy().baseline((None, 0))
"""
# ---- pick the two conditions from your Epochs ----
epochs_std_full = orig_MMN_merged_epoch_fif[std_key].copy()
epochs_dev_full = orig_MMN_merged_epoch_fif[dev_key].copy()

# baseline *method* (not attribute), then crop
epochs_std_full.apply_baseline(baseline=(None, 0))
epochs_dev_full.apply_baseline(baseline=(None, 0))

# Evoked (ERP) and MMN difference
ev_std = epochs_std_full.average(picks=channel)
ev_dev = epochs_dev_full.average(picks=channel)
mmn_ev = mne.combine_evoked([ev_dev, ev_std], weights=[1, -1])  # dev - std

# Trial-wise arrays for SNR
X_std = epochs_std_full.get_data(picks=channel)[:, 0, :]  # (n_epochs, n_times)
X_dev = epochs_dev_full.get_data(picks=channel)[:, 0, :]  # (n_epochs, n_times)

times = ev_std.times
win   = (times >= mmn_window[0]) & (times <= mmn_window[1])

# ERP-SNR definition: mean(|ERP|) / std(trials) over the window
erp_std = ev_std.data[0, :]  # (n_times,)
erp_dev = ev_dev.data[0, :]
erp_mmn = mmn_ev.data[0, :]

noise_std = X_std.std(axis=0, ddof=1)
noise_dev = X_dev.std(axis=0, ddof=1)
# For MMN noise, use dev minus mean standard per trial:
X_mmn = X_dev - X_std.mean(axis=0, keepdims=True)  # (n_dev, n_times)
noise_mmn = X_mmn.std(axis=0, ddof=1)

EPS = 1e-12
snr_std_win = (np.abs(erp_std[win]).mean()) / (noise_std[win].mean() + EPS)
snr_dev_win = (np.abs(erp_dev[win]).mean()) / (noise_dev[win].mean() + EPS)
snr_mmn_win = (np.abs(erp_mmn[win]).mean()) / (noise_mmn[win].mean() + EPS)

# ----------- PLOT -----------
plt.figure(figsize=(10, 5))
plt.plot(times, erp_std, color='#1f77b4', label=f'{std_key}  (SNR={snr_std_win:.2f})')
plt.plot(times, erp_dev, color='#d62728', label=f'{dev_key}  (SNR={snr_dev_win:.2f})')
plt.plot(times, erp_mmn, color='black',   label=f'Dev−Std  (SNR={snr_mmn_win:.2f})', linewidth=2)

# shade MMN window
plt.axvspan(mmn_window[0], mmn_window[1], color='gray', alpha=0.15, label='MMN window')

plt.axhline(0, color='k', linewidth=0.8, alpha=0.6)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (µV)')
plt.title(f'ERP at {channel} with MMN Window and ERP‑SNR')
# de-duplicate legend
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(dict(zip(labels, handles)).values(), dict(zip(labels, handles)).keys(), loc='best', fontsize=9)
plt.grid(True, ls='--', alpha=0.3)
plt.tight_layout()
plt.show()


Reading /Users/neuroling/Downloads/20250610_EEG-Demo/rawfif_all/MMN_epochs/new_MMN_merged_noFiltSamp_epochs.fif ...


/var/folders/dt/yj00f34n5x52_p314tnwfjkw0000gn/T/ipykernel_53445/1299727441.py:15: RuntimeWarning: This filename (/Users/neuroling/Downloads/20250610_EEG-Demo/rawfif_all/MMN_epochs/new_MMN_merged_noFiltSamp_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  orig_MMN_merged_epoch_fif = mne.read_epochs(epoch_DIR / Path('new_MMN_merged_noFiltSamp_epochs.fif'), preload=True)


    Found the data of interest:
        t =     -99.61 ...     700.20 ms
        0 CTF compensation matrices available
Not setting metadata
1000 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
